In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from pprint import pprint
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier # rf분류기

In [2]:
rf_basic = pd.read_csv('basic_rf_importance.csv')
rf_hard = pd.read_csv('fine_rf_importance.csv')

In [14]:
rf_hard.head(50)

,importance,feature
0,0.069372,get_money_Count_week_stat
1,0.052033,play_time_by_cnt_dt
2,0.035559,pattern_retained
3,0.023651,get_money_MA_1_time_series
4,0.020949,game_combat_time_diff_w8_by_last_time_series
5,0.020890,item_hongmun_mean_basic_time
6,0.020766,item_hongmun_sum_stat
7,0.012710,play_time_median_stat
8,0.012412,get_money_sum_stat
9,0.010675,cnt_dt_var_time_series


In [15]:
rf_basic.head(50)

,importance,feature
0,0.016006,play_time_by_cnt_dt
1,0.013042,play_time_MA_5_time_series
2,0.011742,play_time_sum_stat
3,0.011369,play_time_max_stat
4,0.010975,play_time_median_stat
5,0.010966,play_time_var_time_series
6,0.010027,play_time_mean_time_series
7,0.009952,"('cnt_dt', 'play_time')"
8,0.009563,play_time_min_stat
9,0.009515,play_time_diff_w8_by_last_time_series
